In [29]:
# !wget https://lazyprogrammer.me/course_files/nlp/tmdb_5000_movies.csv

In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import json

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [32]:
data = pd.read_csv("tmdb_5000_movies.csv")

In [33]:
data.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [34]:
def get_name_from_json(x):
  return ' '.join([''.join(j['name']) for j in json.loads(x)])

In [35]:
cols_with_json = ['genres','keywords','production_companies','production_countries','spoken_languages']

In [36]:
for col in cols_with_json:
  data[col] = data[col].apply(get_name_from_json)

In [65]:
data.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,string
0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,Ingenious Film Partners Twentieth Century Fox ...,...,2009-12-10,2787965087,162.0,English Español,Released,Enter the World of Pandora.,Avatar,7.2,11800,Action Adventure Fantasy Science Fiction cultu...
1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,Walt Disney Pictures Jerry Bruckheimer Films S...,...,2007-05-19,961000000,169.0,English,Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Adventure Fantasy Action ocean drug abuse exot...
2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6 bri...,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,Columbia Pictures Danjaq B24,...,2015-10-26,880674609,148.0,Français English Español Italiano Deutsch,Released,A Plan No One Escapes,Spectre,6.3,4466,Action Adventure Crime spy based on novel secr...
3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,Legendary Pictures Warner Bros. DC Entertainme...,...,2012-07-16,1084939099,165.0,English,Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Action Crime Drama Thriller dc comics crime fi...
4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,Walt Disney Pictures,...,2012-03-07,284139100,132.0,English,Released,"Lost in our world, found in another.",John Carter,6.1,2124,Action Adventure Science Fiction based on nove...


In [38]:
data['string'] = data['genres'] +" "+ data['keywords']

In [39]:
data['string']

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states–mexico bar...
4799                                      Comedy Romance 
4800    Comedy Drama Romance TV Movie date love at fir...
4801                                                     
4802     Documentary obsession camcorder crush dream girl
Name: string, Length: 4803, dtype: object

In [40]:
tfidf = TfidfVectorizer(max_features=2000)
X = tfidf.fit_transform(data["string"])

#### Steps to create movie recomendation system
- Create a movie string with genre and Keywords
- Convert the string to vector using tfidf
- now we have X metrix
- Check the similarty of values in X for with the watched movie metrix

In [44]:
# How to Create watched movie metrics
moviesidx = pd.Series(data.index, index = data['title'])

In [66]:
idx = moviesidx['Avatar']

In [67]:
query = X[idx]

In [68]:
scores = cosine_similarity(query,X).flatten()

In [69]:
#arg sort will sort the indices based on values
scores.argsort()
# get the lowest score of cosine as they are the similer distance
# get the first six scores
recomended_idx = (-scores).argsort()[:6]
top_similer_movies = moviesidx[recomended_idx]
print(top_similer_movies)

title
Avatar                   0
Silent Running        4332
Alien³                 838
Aliens                2403
Alien                 3158
Planet of the Apes     278
dtype: int64


In [70]:
recomended_idx

array([   0, 4332,  838, 2403, 3158,  278])

In [62]:
import numpy as np

a = np.array([1,4,2,5])
a.argsort()

array([0, 2, 1, 3])